In [1]:
!pip install ultralytics
!pip install albumentations==1.0.3
!pip install ipywidgets

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/f4/79/b192c64e13eefc7f8543a10080767ab057850fa9bdbc38e7c261a3432c93/ultralytics-8.0.227-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.3.1
    Uninstalling albumentations-1.3.1:
      Successfully uninstalled albumentations-1.3.1


In [4]:
from tqdm.auto import tqdm

import numpy as np

import os
import cv2


def tile(bounds, x_start, y_start, size):
    _class, x_min, y_min, x_max, y_max = bounds
    x_min, y_min, x_max, y_max = x_min - x_start, y_min - y_start, x_max - x_start, y_max - y_start

    if (x_min > size) or (x_max < 0.0) or (y_min > size) or (y_max < 0.0):
        return None
    
    x_max_trunc = min(x_max, size) 
    x_min_trunc = max(x_min, 0)

    if x_max == x_min or y_max == y_min: return None

    if (x_max_trunc - x_min_trunc) / (x_max - x_min) < 0.3:
        return None

    y_max_trunc = min(y_max, size) 
    y_min_trunc = max(y_min, 0) 
    if (y_max_trunc - y_min_trunc) / (y_max - y_min) < 0.3:
        return None
        
    x_center = (x_min_trunc + x_max_trunc) / 2.0 / size
    y_center = (y_min_trunc + y_max_trunc) / 2.0 / size
    x_extend = (x_max_trunc - x_min_trunc) / size
    y_extend = (y_max_trunc - y_min_trunc) / size
    
    return (_class, x_center, y_center, x_extend, y_extend)


# https://stackoverflow.com/a/64097592/16660603
def yolobbox2bbox(coords: list, size: int) -> list:
    _c, x, y, w, h = coords

    x1 = int((x - w / 2) * size)
    x2 = int((x + w / 2) * size)
    y1 = int((y - h / 2) * size)
    y2 = int((y + h / 2) * size)
    
    if x1 < 0:
        x1 = 0
    if x2 > size - 1:
        x2 = size - 1
    if y2 < 0:
        y1 = 0
    if y2 > size - 1:
        y2 = size - 1

    return [int(_c), x1, y1, x2, y2]


def load_annotation(path: str) -> list:
    with open(path, 'r') as f:
        ann = [list(map(float, line.replace('\n', '').split())) for line in f]

    return ann


def load_img_path(load_data_path: str, img_path: str, labels_path: str) -> tuple:
    img_paths, labels_paths = [], []
    img, lbl = os.listdir(img_path), os.listdir(labels_path)
    img.sort(); lbl.sort()

    with open(load_data_path, 'r') as f:
        for name in f:
            name = name.strip()
            name = name.replace('\n', '') + '.jpg'
            if name in img: img_paths.append(f'{img_path}/{name}')
            else: continue

            name = name.replace(".jpg", ".txt")
            if name in lbl: labels_paths.append(f'{labels_path}/{name}')
            else: img_paths.remove(f'{img_path}/{name}') # Fail safe
    
    return (img_paths, labels_paths)


def process_data(mode: str, load_data_list: tuple, save_path: str) -> None:
    img, label = load_img_path(*load_data_list)
    assert len(img) == len(label), 'Image and label length mismatch'

    default_size = 800
    tile_size = 512
    tile_overlap = 64

    output_paths = [save_path + f'/{mode}/images', save_path + f'/{mode}/labels']
    for _path in output_paths:
        if not os.path.isdir(_path):
            os.makedirs(_path)

    for i in tqdm(range(len(img))):
        image_path = img[i]
        label_path = label[i]

        _img_name = os.path.basename(image_path)
        _annot_name = os.path.basename(label_path)

        image = cv2.imread(image_path)
        image = cv2.resize(image, (default_size, default_size))
        size = image.shape[0]

        annotation_list = load_annotation(label_path)
        coords_list = [yolobbox2bbox(al, size=size) for al in annotation_list]

        x_tiles = (size + tile_size - tile_overlap - 1) // (tile_size - tile_overlap)
        y_tiles = (size + tile_size - tile_overlap - 1) // (tile_size - tile_overlap)

        for x in range(x_tiles):
            for y in range(y_tiles):
                x_end = min((x + 1) * tile_size - tile_overlap * (x != 0), size)
                x_start = x_end - tile_size
                y_end = min((y + 1) * tile_size - tile_overlap * (y != 0), size)
                y_start = y_end - tile_size

                save_tile_path = output_paths[0] + f'/{_img_name.split(".")[0]}_{x_start}_{y_start}.jpg'
                save_label_path = output_paths[1] + f'/{_annot_name.split(".")[0]}_{x_start}_{y_start}.txt'

                cut_tile = np.zeros(shape=(tile_size, tile_size, 3), dtype=np.uint8)
                cut_tile[0:tile_size, 0:size, :] = image[y_start:y_end, x_start:x_end, :]
                cv2.imwrite(save_tile_path, cut_tile)

                found_tags = [
                    tile(bounds, x_start, y_start, tile_size)
                    for bounds in coords_list]
                found_tags = [el for el in found_tags if el is not None]

                with open(save_label_path, 'w+') as f:
                    for tags in found_tags:
                        f.write(' '.join(str(x) for x in tags) + '\n')


data_path = '../input/diordata'
save_path = '/kaggle/temp'

train_img_name_path = data_path + '/ImageSets/train.txt'
val_img_name_path = data_path + '/ImageSets/val.txt'
test_img_name_path = data_path + '/ImageSets/test.txt'

img_path = data_path + '/images'
labels_path = data_path + '/labels'

#Train
process_data('train', (train_img_name_path, img_path, labels_path), save_path)
# Val
process_data('val', (val_img_name_path, img_path, labels_path), save_path)
# Test
process_data('test', (test_img_name_path, img_path, labels_path), save_path)

  0%|          | 0/19004 [00:00<?, ?it/s]

  0%|          | 0/2112 [00:00<?, ?it/s]

  0%|          | 0/2347 [00:00<?, ?it/s]

In [9]:
from ultralytics import YOLO

import yaml

model_data_path = '/kaggle/working/Model'

model = YOLO('yolov8n.yaml')
labels = [
    'Storage Tank',
    'Baseball field',
    'Tennis court',
    'Basketball Court',
    'Wind mill',
    'Vehicle',
    'Harbor',
    'Ship',
    'Airplane',
    'Bridge',
    'Overpass',
    'Expressway toll station',
    'Train station',
    'Chimney',
    'Ground Track Field',
    'Dam',
    'Expressway service area',
    'Stadium',
    'Airport',
    'Golf course'
]
config = {
    'path': '/kaggle/temp',
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'names': {str(i): label for i, label in enumerate(labels)}
}

with open(model_data_path + '\config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

model.train(data=model_data_path + '\config.yaml', imgsz=512, epochs=20, batch=64, name='Yolov8_3')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning /kaggle/temp/train/labels.cache... 76016 images, 4889 backgrounds, 0 corrupt: 100%|██████████| 76016/76016 [00:00<?, ?it/s]

train: WARNING ⚠️ /kaggle/temp/train/images/00594_288_0.jpg: 1 duplicate labels removed


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/temp/val/labels.cache... 8448 images, 535 backgrounds, 0 corrupt: 100%|██████████| 8448/8448 [00:00<?, ?it/s]


...

                   all       8448      36967      0.831      0.746      0.813      0.576

20 epochs completed in 3.716 hours.
Optimizer stripped from runs/detect/Yolov8_35/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/Yolov8_35/weights/best.pt, 6.2MB

Validating runs/detect/Yolov8_35/weights/best.pt...
Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
YOLOv8n summary (fused): 168 layers, 3009548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 66/66 [01:08<00:00,  1.03s/it]


                   all       8448      36967       0.83      0.746      0.813      0.576
          Storage Tank       8448       5691       0.88      0.719      0.806      0.491
        Baseball field       8448        935      0.871      0.913      0.948      0.804
          Tennis court       8448       2237      0.926      0.913      0.954      0.823
      Basketball Court       8448        713      0.874      0.887      0.923      0.792
             Wind mill       8448        872      0.891      0.768       0.84      0.445
               Vehicle       8448       6955      0.819      0.487      0.619       0.34
                Harbor       8448       1026      0.718      0.636      0.685      0.447
                  Ship       8448      11155      0.897       0.93      0.952      0.566
              Airplane       8448       1917      0.918      0.905      0.935      0.661
                Bridge       8448        857      0.796      0.447      0.568      0.333
              Overpas

lr/pg0,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg1,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg2,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▁▃▄▅▆▆▇▇▇▇██████████
metrics/mAP50-95(B),▁▃▄▅▅▆▆▇▇▇▇█████████
metrics/precision(B),▁▁▄▅▅▆▇▇▇▇██████████
metrics/recall(B),▁▃▄▅▆▆▇▇▇▇▇█████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▄▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d8930705f00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043